## Ironsworn

[Official site.](https://www.ironswornrpg.com/)

Roll 1d6 + modifier (action die) against two d10s (challenge dice).

* If the action die is > both challenge dice, it's a strong hit.
* If it is > than one challenge die, it's a weak hit.
* Otherwise, it's a miss.

Additionally, there is a momentum score.

* A positive momentum can be used to zero out any challenge die less than its value.
* A negative momentum zeroes out any action die equal to its value.

In [1]:
import piplite
await piplite.install("hdroller")

import hdroller

## The classic mistake in implementing *Ironsworn*

Let's start without momentum and add it later.

The classic mistake with finding probabilities for *Ironsworn* is that this doesn't work:

In [2]:
def does_not_work(modifier):
    return 2 @ (hdroller.d6 + modifier > hdroller.d10)

print(does_not_work(1))

Denominator: 3600
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       0 |   1521 |  42.250000% |
|       1 |   1638 |  45.500000% |
|       2 |    441 |  12.250000% |



The trouble with this is that it effectively rolls two independent contests of d6 + modifier against d10, not using the same d6 against both.
This overestimates the chance of the central outcome (weak hit).

## Correct ways of implementing *Ironsworn*

A correct way to do this would be to use the `apply()` function to the three dice:

In [3]:
def using_apply(modifier):
    def func(a, c1, c2):
        return (a > c1) + (a > c2)
    return hdroller.apply(func, hdroller.d6 + modifier, hdroller.d10, hdroller.d10)

print(using_apply(1))

Denominator: 600
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       0 |    271 |  45.166667% |
|       1 |    238 |  39.666667% |
|       2 |     91 |  15.166667% |



Or you can "roll" the action die first, and then use the `sub()` method to compare the result to two d10s.

In [4]:
def using_sub(modifier):
    return (hdroller.d6 + modifier).sub(lambda a: 2 @ (a > hdroller.d10))

print(using_sub(1))

Denominator: 600
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       0 |    271 |  45.166667% |
|       1 |    238 |  39.666667% |
|       2 |     91 |  15.166667% |



## Adding momentum

Now let's add the momentum mechanic.

We can do this by modifying the action and challenge dice, which can also be done using the `sub()` method.

In [8]:
def using_sub(modifier, momentum):
    action = hdroller.d6
    challenge = hdroller.d10
    if momentum > 1:
        challenge = challenge.sub(lambda c: 0 if c < momentum else c)
    if momentum < 0:
        # You can use a dict to map old outcomes to new outcomes.
        # Any outcomes not mentioned are preserved.
        action = action.sub({-momentum: 0})
    return (action + modifier).sub(lambda a: 2 @ (a > challenge))

print(using_sub(1, 5))

Denominator: 600
| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       0 |    185 |  30.833333% |
|       1 |    290 |  48.333333% |
|       2 |    125 |  20.833333% |

